# GeoSPARQL Examples

In the following tutorial we will show how to ETL geometric data into an AllegroGraph repository, and then demonstrate a few GeoSPARQL queries. This notebook is not meant to be an exhaustive list of every possible SPARQL GeoSPARQL function, but to show the patterns that all subsequent functions follow. There are many more functions available within the Franz SPARQL engine than we will show, and more can be found [here](https://opengeospatial.github.io/ogc-geosparql/geosparql11/geo.ttl).

We start by importing `geopandas` and some utility functions from the agraph-python `franz` package, and connecting to a new repository. Make sure to add your own connection parameters:

In [ ]:
import geopandas as gpd

from franz.openrdf.connect import ag_connect
from franz.openrdf.vocabulary import RDF, RDFS

import json
import shortuuid
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt

#the repository connection. Please add your connection parameters here
conn = ag_connect('geosparql-examples', clear=True,
                   host='localhost', port='10035',
                   user='your AllegroGraph username', password='your AllegroGraph password')

#defining namespaces
geo = conn.namespace('http://www.opengis.net/ont/geosparql#')
conn.setNamespace('geo', 'http://www.opengis.net/ont/geosparql#')

uri = 'http://franz.com/'
f = conn.namespace(uri)
conn.setNamespace('f', uri)

## ETL

Now we read in some data that contains various geometric objects. We will look at Provinces and Municipalities in the Netherlands (and make some simple plots of the data). If you are interested you can download more data here at [https://gadm.org/download_country.html](https://gadm.org/download_country.html). Importing a file with `geopandas` makes it so that the geometry data is correctly added in the dataframe.

In [ ]:
df_province = gpd.read_file('https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_NLD_1.json.zip')
df_province.head()

In [ ]:
df_province.plot()
plt.show()

As you can see, the last column named `geometry` contains a number of **Multipolygons**.

Before we are able to ETL the data into our repository, we need to define a function that converts each entry in the `geometry` column into a **MultiPolygon  geo:geoJSONLiteral**. You can read more about the shapes of these [here](https://datatracker.ietf.org/doc/html/rfc7946#section-3.1.7):


In [ ]:
def convert_geo_to_literal(geometry_cell):
    _json = {"type": "MultiPolygon", "coordinates": []}
    for geom in geometry_cell.geoms:
        multi = []
        for point in geom.exterior.coords:
            coordinates = [point[0], point[1]]
            multi.append(coordinates)
        _json["coordinates"].append([multi])
    json_string = json.dumps(_json)
    return conn.createLiteral(json_string, datatype=geo.geoJSONLiteral)

We will do a very simplified version of the ETL. We could add more metadata about each province, but for now we will only gather the names and rdf:types of each geometric feature (a `geo:Feature` being the object we care about, in this case a province). The most important thing is that you need to create an object with `rdf:type` `geo:Feature`, and also create another objects with `rdf:type` `geo:Geometry`. Then that geometry object has the geometric data attached as a literal.

In [ ]:
triples = []
for i in range(df_province.shape[0]):
    province_uri = conn.createURI(f"{uri}{df_province['GID_1'][i]}")
    triples.append((province_uri, RDF.TYPE, conn.createURI(f"{uri}{df_province['ENGTYPE_1'][i]}")))
    triples.append((province_uri, RDF.TYPE, geo.Feature))

    #add name info and a label
    triples.append((province_uri, RDFS.LABEL, df_province['NAME_1'][i]))
    triples.append((province_uri, f.provinceName, df_province['NAME_1'][i]))

    #add geometry
    geo_uri = conn.createURI(f"{uri}{shortuuid.uuid()}")
    triples.append((geo_uri, RDF.TYPE, geo.Geometry))
    triples.append((province_uri, geo.hasGeometry, geo_uri))
    geometry_literal = convert_geo_to_literal(df_province['geometry'][i])
    triples.append((geo_uri, geo.asGeoJSON, geometry_literal))
    
conn.addTriples(triples)
conn.deleteDuplicates(mode='spo')

Now we will also ETL some more Netherlands Municipality Data

In [ ]:
df_muni = gpd.read_file('https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_NLD_2.json.zip')
df_muni.head()

In [ ]:
df_muni.plot()
plt.show()

We will skip bodies of water for simplicity's sake

In [ ]:
triples = []
for i in range(df_muni.shape[0]):
    if df_muni['ENGTYPE_2'][i] != 'Waterbody':
        muni_uri = conn.createURI(f"{uri}{df_muni['GID_2'][i]}")
        triples.append((muni_uri, RDF.TYPE, conn.createURI(f"{uri}{df_muni['ENGTYPE_2'][i]}")))
        triples.append((muni_uri, RDF.TYPE, geo.Feature))

        #get name info
        triples.append((muni_uri, RDFS.LABEL, df_muni['NAME_2'][i]))
        triples.append((muni_uri, f.municipalityName, df_muni['NAME_2'][i]))

        #link to provinces
        province_uri = conn.createURI(f"{uri}{df_muni['GID_1'][i]}")
        triples.append((province_uri, f.municipality, muni_uri))

        #add geometry
        geo_uri = conn.createURI(f"{uri}{shortuuid.uuid()}")
        triples.append((geo_uri, RDF.TYPE, geo.Geometry))
        triples.append((muni_uri, geo.hasGeometry, geo_uri))
        geometry_literal = convert_geo_to_literal(df_muni['geometry'][i])
        triples.append((geo_uri, geo.asGeoJSON, geometry_literal))
conn.addTriples(triples)
conn.deleteDuplicates(mode='spo')

We can examine the data in Gruff. Notice that the Provinces and Municipalities all have related `geometry` objects and to that we connect the **Multipolygon** literal

![sample layout](img/geosparql-etl-layout.png)

## The format of GeoSPARQL queries

GeoSPARQL queries do comparisons and calculations between different geometries. Those can be points, lines, polygons and others. However, most queries will follow the following format.

```
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?result WHERE {
    ?feature1 a geo:Feature ;
              geo:hasGeometry ?geometry1 .
    ?geometry1 a geo:Geometry ;
              geo:asGeoJSON ?geoJSON1 .
              
    ?feature2 a geo:Feature ;
              geo:hasGeometry ?geometry2 .
    ?geometry2 a geo:Geometry ;
              geo:asGeoJSON ?geoJSON2 .
              
    BIND ( geof:sfWithin(?geometry1, ?geometry2) as ?result ) .
```

Instead of `geof:sfWithin` can be replaced by many of the geospatial functions.

## The **S** imple **F** eatures functions: 

As defined by Simple Features [[OGCSFACA]](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#OGCSFACA) [[ISO19125-1]](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#ISO19125-1), AllegroGraph supports `geof:sfEquals`, `geof:sfDisjoint`, `geof:sfIntersects`, `geof:sfTouches`, `geof:sfCrosses`, `geof:sfWithin`, `geof:sfContains` and `geof:sfOverlaps` as SPARQL extension functions, consistent with their corresponding DE-9IM intersection patterns, where geof: namespace has uri http://www.opengis.net/def/function/geosparql/. Here are a few examples of those:


## sfWithin / sfContains

In the following example we find which municipality a given point is within.

As a note, `sfWithin` is the inverse of `sfContains`. The key difference is directionality
* `sfWithin` is used when you want to check if one geometry is completely inside another.
* `sfContains` is used when you want to check if one geometry completely encompasses another.

In [ ]:
%%time
query_string = """
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?label  WHERE {
  BIND ( 'POINT(4.9041 52.3676)'^^geo:wktLiteral  as ?point) #The point
  ?municipality geo:hasGeometry ?geometry ;
                a f:Municipality ;
                rdfs:label ?label .
  ?geometry geo:asGeoJSON ?geoJsonLiteral .
  BIND ( geof:sfWithin( ?point, ?geoJsonLiteral ) as ?result ) #the function
  FILTER ( ?result = "true"^^xsd:boolean ) . }"""
result = conn.executeTupleQuery(query_string).toPandas()
result.head()

We will plot to show where the point lies

In [ ]:
from shapely.geometry import Point
from shapely import wkt

point_wkt = 'POINT(4.9041 52.3676)'  # Amsterdam coordinates
point = wkt.loads(point_wkt)
gdf_point = gpd.GeoSeries([point])
ax = df_muni.plot(color='lightblue', edgecolor='black', figsize=(10, 10))
gdf_point.plot(ax=ax, color='red', marker='o', markersize=100)

# Display the plot
plt.show()

## sfIntersects

We find which Provinces a certain line intersects. The line starts at the top of the netherlands and ends at the bottom.

In [ ]:
%%time
query_string = """
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?label  WHERE {
  BIND ( '''{
              "type": "LineString",
              "coordinates": [
                    [6.074182, 53.510403],  
                    [5.887200, 50.750383] 
                ] }'''^^geo:geoJSONLiteral  as ?line )
  ?province geo:hasGeometry ?geometry ;
                a f:Province ;
                rdfs:label ?label .
  ?geometry geo:asGeoJSON ?geoJsonLiteral .
  BIND ( geof:sfIntersects(?line, ?geoJsonLiteral ) as ?result )
  FILTER ( ?result = "true"^^xsd:boolean ) . }"""
result = conn.executeTupleQuery(query_string).toPandas()
result.head()

We will prove by plotting the line over the multipolygons

In [ ]:
from shapely.geometry import LineString

line = LineString([
    [6.074182, 53.510403],  # Top of the Netherlands
    [5.887200, 50.750383]   # Bottom of the Netherlands
])

gdf_line = gpd.GeoSeries([line])
ax = df_province.plot(color='lightblue', edgecolor='black', figsize=(10, 10))
gdf_line.plot(ax=ax, color='red', linewidth=2)
plt.show()

### Distance

We start by computing the shortest **distance** between two geometries. In this example we will compute the **distance** between 'Amsterdam' and 'Groningen' in kilometers. Notice the `qudt` prefix for units of measurement. You can read more about that [here](https://qudt.org/2.1/vocab/unit)

In [ ]:
%%time
query_string = """
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX qudt: <http://qudt.org/vocab/unit/>

SELECT (geof:distance(?amsterdamGeoLit, ?groningenGeoLit, qudt:KiloM) as ?distance) WHERE {
    #amsterdam
    ?amsterdam f:municipalityName "Amsterdam" ;
               geo:hasGeometry ?amsterdamGeo .
    ?amsterdamGeo geo:asGeoJSON ?amsterdamGeoLit .

    #Gronginen
    ?groningen f:municipalityName "Groningen" ;
               geo:hasGeometry ?groningenGeo .
    ?groningenGeo geo:asGeoJSON ?groningenGeoLit . }"""
result = conn.executeTupleQuery(query_string).toPandas()
result.head()

## Intersection

In the following query we show that there is an intersection between 'Amsterdam' and the province it is in 'Noord-Holland'. Since 'Amsterdam' is in 'Noord-Holland' it makes sense that we get a full **Multipolygon** as a result.

In [ ]:
query_string = """
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT (geof:intersection(?amsterdamGeoLit, ?nhGeoLit) as ?intersection) WHERE {
    #amsterdam
    ?amsterdam f:municipalityName "Amsterdam" ;
               geo:hasGeometry ?amsterdamGeo .
    ?amsterdamGeo geo:asGeoJSON ?amsterdamGeoLit .

    #Noord-Holland
    ?nh f:provinceName "Noord-Holland" ;
               geo:hasGeometry ?nhGeo .
    ?nhGeo geo:asGeoJSON ?nhGeoLit . }"""
result = conn.executeTupleQuery(query_string).toPandas()
pprint(result['intersection'][0])

## Union

Here we run a query that returns a **Multipolygon** of the *union* of the two geometries. Since these two are not connected it should return the two multipolygons of each, as one multipolygon.

In [ ]:
query_string = """
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX uom: <http://www.opengis.net/def/uom/OGC/1.0/>

SELECT (geof:union(?amsterdamGeoLit, ?groningenGeoLit) as ?union) WHERE {
    #amsterdam
    ?amsterdam f:municipalityName "Amsterdam" ;
               geo:hasGeometry ?amsterdamGeo .
    ?amsterdamGeo geo:asGeoJSON ?amsterdamGeoLit .

    #Gronginen
    ?groningen f:municipalityName "Groningen" ;
               geo:hasGeometry ?groningenGeo .
    ?groningenGeo geo:asGeoJSON ?groningenGeoLit . }"""
result = conn.executeTupleQuery(query_string).toPandas()
pprint(result['union'][0])